# A Transfer Learning and Optimized CNN Based Intrusion Detection System for Internet of Vehicles 
This is the code for the paper entitled "**A Transfer Learning and Optimized CNN Based Intrusion Detection System for Internet of Vehicles**" accepted in IEEE International Conference on Communications (IEEE ICC).  
Authors: Li Yang (lyang339@uwo.ca) and Abdallah Shami (Abdallah.Shami@uwo.ca)  
Organization: The Optimized Computing and Communications (OC2) Lab, ECE Department, Western University

**Notebook 2: CNN Model Development**  
Aims:  
&nbsp; 1): Generate training and test images  
&nbsp; 2): Construct CNN models (a CNN model by own, Xception, VGG16, VGG19, Resnet, Inception, InceptionResnet)  
&nbsp; 3): Tune the hyperparameters of CNN models (hyperparameter optimization)  

## Import libraries

In [1]:
import os
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,GlobalAveragePooling2D,Input,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.applications.xception import  Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import  ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.mobilenet import MobileNet
import tensorflow.keras.callbacks as kcallbacks
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import math
import random
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from PIL import Image
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

2022-07-29 08:44:57.007601: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Generate Training and Test Images

In [2]:
#generate training and test images
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)
BATCHSIZE=128	#could try 128 or 32

#Normalization
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train_224/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        './test_224/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

Found 2076 images belonging to 5 classes.
Found 518 images belonging to 5 classes.


### Define the image plotting functions

In [3]:
import pandas as pd
import time
output_df = pd.DataFrame(columns=['Loss','Accuracy',  'Val-Loss', 'Val-Accuracy', 'Time'])
output_index = list()
timelist=list()

In [4]:
#plot the figures
class LossHistory(keras.callbacks.Callback):
    def __init__(self, model_name):
        self.model_name=model_name
    def on_train_begin(self, logs={}):
        
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}
    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('accuracy'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_accuracy'))
    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('accuracy'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_accuracy'))
    
    #把数据存入df中
    def on_train_end(self, logs={}):
        global output_df
        global output_index
        idx=0
        maxValue=0
       
        for i in range(len(self.val_acc['epoch'])):
            if self.val_acc['epoch'][i]>maxValue :
                maxValue=self.val_acc['epoch'][i]
                idx=i
        
        result_dict={
            'Loss': self.losses['epoch'][idx],
            'Accuracy':self.accuracy['epoch'][idx],
            'Val-Loss':self.val_loss['epoch'][idx],
            'Val-Accuracy':self.val_acc['epoch'][idx],
        }
        output_df=output_df.append(result_dict,ignore_index=True)
        output_index.append(self.model_name)    
    def loss_plot(self, loss_type): 
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # acc
            plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
            # loss
            plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()

In [5]:
history_CNN= LossHistory("CNN")
history_Xception= LossHistory("Xception")
history_Inception= LossHistory("Inception")
history_VGG19= LossHistory("VGG19")
history_Resnet= LossHistory("Resnet")
history_InceptionResnet= LossHistory("InceptionResnet")
history_VGG16= LossHistory("VGG16")


# Construct CNN models

### Model 1: a CNN model by own (baseline)

In [6]:
def cnn_by_own(input_shape,num_class,epochs,savepath='./model_own.h5'):
    model = Sequential()
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=input_shape,padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(num_class,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    #train model
    earlyStopping=kcallbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, mode='auto')
    saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
    hist=model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        callbacks=[earlyStopping,saveBestModel,history_CNN],
    )
    

In [7]:

start_time=time.time()
cnn_by_own(input_shape=INPUT_SIZE,num_class=5,epochs=20)
end_time=time.time()
timelist.append(end_time-start_time)


2022-07-29 08:45:00.190193: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-07-29 08:45:00.238404: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:ce:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-07-29 08:45:00.238450: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-07-29 08:45:00.245688: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-07-29 08:45:00.245834: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-07-29 08:45:00.247190: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcu

Epoch 1/20


2022-07-29 08:45:03.864608: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-07-29 08:45:04.713278: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2022-07-29 08:45:05.885547: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-07-29 08:45:06.602253: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-07-29 08:45:09.298118: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


17/17 [==============================] - 30s 1s/step - loss: 1.3813 - accuracy: 0.6349 - val_loss: 1.0664 - val_accuracy: 0.6834

Epoch 00001: val_accuracy improved from -inf to 0.68340, saving model to ./model_own.h5
Epoch 2/20
17/17 [==============================] - 18s 1s/step - loss: 1.1069 - accuracy: 0.6599 - val_loss: 0.9659 - val_accuracy: 0.6834

Epoch 00002: val_accuracy did not improve from 0.68340
Epoch 3/20
17/17 [==============================] - 18s 1s/step - loss: 0.9854 - accuracy: 0.6599 - val_loss: 0.7934 - val_accuracy: 0.6834

Epoch 00003: val_accuracy did not improve from 0.68340
Epoch 00003: early stopping


/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of a CNN by own: 99.884%

### Model 2: Xception

In [8]:
def xception( num_class, epochs,savepath='./xception.h5',history=history_Xception,input_shape=INPUT_SIZE):
    model_fine_tune = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:131]:		#could be tuned to be 50, 100, or 131
        layer.trainable = False
    for layer in model_fine_tune.layers[131:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='xception')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=3, verbose=1, mode='auto')	#patience could be tuned by 2 and 3
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
    )


In [9]:
#default only 50, tf36cnn 99
start_time=time.time()
xception(num_class=5,epochs=20)
end_time=time.time()
timelist.append(end_time-start_time)

# Insufficient Video Memory 显存不足

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of b0042744bf5b25fce3cb969f33bebb97 so we will re-download the data.
83689472/83683744 [==============================] - 2s 0us/step


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
17/17 [==============================] - 23s 1s/step - loss: 0.5765 - accuracy: 0.7914 - val_loss: 0.1309 - val_accuracy: 0.9691

Epoch 00001: val_accuracy improved from -inf to 0.96911, saving model to ./xception.h5


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
17/17 [==============================] - 17s 975ms/step - loss: 0.1271 - accuracy: 0.9725 - val_loss: 0.0460 - val_accuracy: 0.9942

Epoch 00002: val_accuracy improved from 0.96911 to 0.99421, saving model to ./xception.h5
Epoch 3/20
17/17 [==============================] - 18s 1s/step - loss: 0.0590 - accuracy: 0.9933 - val_loss: 0.0266 - val_accuracy: 0.9981

Epoch 00003: val_accuracy improved from 0.99421 to 0.99807, saving model to ./xception.h5
Epoch 4/20
17/17 [==============================] - 18s 1s/step - loss: 0.0328 - accuracy: 0.9966 - val_loss: 0.0176 - val_accuracy: 0.9981

Epoch 00004: val_accuracy did not improve from 0.99807
Epoch 5/20
17/17 [==============================] - 19s 1s/step - loss: 0.0255 - accuracy: 0.9990 - val_loss: 0.0131 - val_accuracy: 1.0000

Epoch 00005: val_accuracy improved from 0.99807 to 1.00000, saving model to ./xception.h5
Epoch 6/20
17/17 [==============================] - 19s 1s/step - loss: 0.0196 - accuracy: 0.9990 - val_loss

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of Xception: 100.0%

### Model 3: VGG16

In [10]:
def vgg16( num_class, epochs,savepath='./VGG16.h5',history=history_VGG16,input_shape=INPUT_SIZE):
    model_fine_tune = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:15]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[15:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output) #GlobalAveragePooling2D layer to convert the features to a single 1280-element vector per image
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[earlyStopping, saveBestModel, history],
    )


In [11]:
start_time=time.time()
vgg16(num_class=5,epochs=20)	#tf36cnn
end_time=time.time()
timelist.append(end_time-start_time)
# Insufficient Video Memory 显存不足

58892288/58889256 [==============================] - 1s 0us/step


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 21s 1s/step - loss: 1.2882 - accuracy: 0.6671 - val_loss: 0.5469 - val_accuracy: 0.7876

Epoch 00001: val_accuracy improved from -inf to 0.78764, saving model to ./VGG16.h5
Epoch 2/20
17/17 [==============================] - 19s 1s/step - loss: 0.4118 - accuracy: 0.8295 - val_loss: 0.2313 - val_accuracy: 0.9131

Epoch 00002: val_accuracy improved from 0.78764 to 0.91313, saving model to ./VGG16.h5
Epoch 3/20
17/17 [==============================] - 19s 1s/step - loss: 0.2439 - accuracy: 0.8984 - val_loss: 0.1532 - val_accuracy: 0.9363

Epoch 00003: val_accuracy improved from 0.91313 to 0.93629, saving model to ./VGG16.h5
Epoch 4/20
17/17 [==============================] - 18s 1s/step - loss: 0.1359 - accuracy: 0.9403 - val_loss: 0.0533 - val_accuracy: 0.9903

Epoch 00004: val_accuracy improved from 0.93629 to 0.99035, saving model to ./VGG16.h5
Epoch 5/20
17/17 [==============================] - 18s 1s/step - loss: 0.0686 - accuracy: 

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of VGG16: 100.0%

### Model 4: VGG19

In [12]:
def vgg19( num_class, epochs,savepath='./VGG19.h5',history=history_VGG19,input_shape=INPUT_SIZE):
    model_fine_tune = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:19]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[19:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[earlyStopping, saveBestModel, history],
    )


In [13]:
start_time=time.time()
vgg19(num_class=5,epochs=20)	#binary classificaiton

end_time=time.time()
timelist.append(end_time-start_time)

80142336/80134624 [==============================] - 2s 0us/step


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 21s 1s/step - loss: 0.8393 - accuracy: 0.7172 - val_loss: 0.1961 - val_accuracy: 0.8880

Epoch 00001: val_accuracy improved from -inf to 0.88803, saving model to ./VGG19.h5
Epoch 2/20
17/17 [==============================] - 18s 1s/step - loss: 0.1394 - accuracy: 0.9422 - val_loss: 0.0115 - val_accuracy: 1.0000

Epoch 00002: val_accuracy improved from 0.88803 to 1.00000, saving model to ./VGG19.h5
Epoch 3/20
17/17 [==============================] - 18s 1s/step - loss: 0.0125 - accuracy: 0.9971 - val_loss: 5.5107e-04 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/20
17/17 [==============================] - 18s 1s/step - loss: 0.0144 - accuracy: 0.9961 - val_loss: 0.0020 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 00004: early stopping


/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of VGG19: 100.0%

### Model 5: ResNet

In [14]:
def resnet( num_class, epochs,savepath='./resnet.h5',history=history_Resnet,input_shape=INPUT_SIZE):
    model_fine_tune = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:120]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[120:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
    )

In [15]:
start_time=time.time()
resnet(num_class=5,epochs=20)	#binary classificaiton
end_time=time.time()
timelist.append(end_time-start_time)

94773248/94765736 [==============================] - 2s 0us/step


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 24s 1s/step - loss: 0.5945 - accuracy: 0.8001 - val_loss: 87.0099 - val_accuracy: 0.1042

Epoch 00001: val_accuracy improved from -inf to 0.10425, saving model to ./resnet.h5


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
17/17 [==============================] - 18s 1s/step - loss: 0.1767 - accuracy: 0.9239 - val_loss: 176.9535 - val_accuracy: 0.1042

Epoch 00002: val_accuracy did not improve from 0.10425
Epoch 3/20
17/17 [==============================] - 18s 1s/step - loss: 0.1068 - accuracy: 0.9634 - val_loss: 193.4737 - val_accuracy: 0.1004

Epoch 00003: val_accuracy did not improve from 0.10425
Epoch 00003: early stopping


/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of Resnet: 98.652%

### Model 6: Inception

In [16]:
def inception( num_class, epochs,savepath='./inception.h5',history=history_Inception,input_shape=INPUT_SIZE):
    model_fine_tune = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:35]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[35:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
    )

In [17]:
start_time=time.time()
inception(num_class=5,epochs=20)	#binary classificaiton

end_time=time.time()
timelist.append(end_time-start_time)

87916544/87910968 [==============================] - 4s 0us/step


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 29s 1s/step - loss: 0.1833 - accuracy: 0.9321 - val_loss: 2.6506 - val_accuracy: 0.6834

Epoch 00001: val_accuracy improved from -inf to 0.68340, saving model to ./inception.h5
Epoch 2/20
17/17 [==============================] - 18s 1s/step - loss: 4.9277e-04 - accuracy: 1.0000 - val_loss: 2.6478 - val_accuracy: 0.1622

Epoch 00002: val_accuracy did not improve from 0.68340
Epoch 3/20
17/17 [==============================] - 19s 1s/step - loss: 4.6998e-05 - accuracy: 1.0000 - val_loss: 2.1890 - val_accuracy: 0.2529

Epoch 00003: val_accuracy did not improve from 0.68340
Epoch 00003: early stopping


/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of Inception: 100.0%

### Model 7: InceptionResnet

In [18]:
def inceptionresnet( num_class, epochs,savepath='./inceptionresnet.h5',history=history_InceptionResnet,input_shape=INPUT_SIZE):
    model_fine_tune = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:500]:	#the number of frozen layers for transfer learning, have tuned from 400-550
        layer.trainable = False
    for layer in model_fine_tune.layers[500:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
    )

In [19]:
start_time=time.time()
inceptionresnet(num_class=5,epochs=20)	# 5-class classificaiton
end_time=time.time()
timelist.append(end_time-start_time)

219062272/219055592 [==============================] - 6s 0us/step


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 37s 1s/step - loss: 0.2732 - accuracy: 0.8979 - val_loss: 3.9568 - val_accuracy: 0.7876

Epoch 00001: val_accuracy improved from -inf to 0.78764, saving model to ./inceptionresnet.h5
Epoch 2/20
17/17 [==============================] - 19s 1s/step - loss: 0.0207 - accuracy: 0.9952 - val_loss: 3.6059 - val_accuracy: 0.7896

Epoch 00002: val_accuracy improved from 0.78764 to 0.78958, saving model to ./inceptionresnet.h5
Epoch 3/20
17/17 [==============================] - 17s 1s/step - loss: 0.0071 - accuracy: 0.9966 - val_loss: 6.9358 - val_accuracy: 0.2703

Epoch 00003: val_accuracy did not improve from 0.78958
Epoch 4/20
17/17 [==============================] - 18s 1s/step - loss: 0.0238 - accuracy: 0.9937 - val_loss: 193.3864 - val_accuracy: 0.1100

Epoch 00004: val_accuracy did not improve from 0.78958
Epoch 00004: early stopping


/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [20]:
import requests
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "模型全部运行完",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


Validation accuracy of InceptionResnet: 99.993%

# Hyperparameter Optimization 
Use VGG16 as an example.  

Tuned hyperparameters of CNN: 
1. The number of frozen layers
2. The number of epochs
3. Early stop patience
4. Learning rate
5. Dropout rate

Hyperparameter optimization methods:
1. Random search
2. Bayesian optimization - Tree Parzen Estimator(BO-TPE)

### 获取最优超参数

In [26]:
import ast
def get_params(str1):
    temp=ast.literal_eval(str1)
    dropout_rate=temp.get('dropout_rate')
    dropout_rate=round(dropout_rate,5)
    num_class=5
    frozen=int(temp.get('frozen'))
    epochs=int(temp.get('epochs'))
    patience=int(temp.get('patience'))
    lr=temp.get('lr')
    lr=round(lr,5)
    frozen=int(temp.get('frozen'))
    verbose=1
    params={
        'num_class': num_class,
        'frozen': frozen,
        'epochs': epochs,
        'patience': patience,
        'lr': lr,
        'dropout_rate' : dropout_rate,
        'verbose': verbose
    }
    return params

### 清除表格中在优化过程中添加的数据

In [27]:
def clear_output_df(index):
    global output_df
    global output_index
    output_df=output_df[0:index] 
    output_index=output_index[:index]
    index+=1
    return index
    

In [28]:
index=7 # 前面有7行 从第七行开始添加
time_opt=list()#存放优化的时间
opt_type=list()
for i in range(7):
    time_opt.append("")
    opt_type.append("")
    


# CNN By Own

In [21]:
def cnn_by_own(input_shape,num_class,epochs=20,patience=2, dropout_rate=0.5,verbose=0,savepath='./model_own.h5',history=history_CNN):
    model = Sequential()
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=input_shape,padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(num_class,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    #train model
    earlyStopping=kcallbacks.EarlyStopping(monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto')
    saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=verbose, save_best_only=True, mode='auto')
    hist=model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        callbacks=[earlyStopping,saveBestModel,history,timer],
        verbose=verbose
    )

In [22]:
def prediction(cnn_model):
#read images from validation folder
    rootdir = './test_224/'
    test_laels = []
    test_images=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
                continue
            test_laels.append(subdir.split('/')[-1])
            test_images.append(os.path.join(subdir, file))

    predict=[]
    length=len(test_images)
    label=validation_generator.class_indices
    label={v: k for k, v in label.items()}
    for i in range(length):
        inputimg=test_images[i]
        test_batch=[]
        thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
        #print(thisimg)
        test_shape=(1,)+thisimg.shape
        thisimg=thisimg.reshape(test_shape)
        cnn_model_batch=cnn_model.predict(thisimg) #use master model to process the input image
        #generate result by model 1
        #
        prob=cnn_model_batch[0,np.argmax(cnn_model_batch,axis=1)[0]]
        res=label[np.argmax(cnn_model_batch,axis=1)[0]]
        predict.append(res)
    acc=accuracy_score(test_laels,predict)
    return acc

In [23]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'lr': abs(float(params['lr'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    frozen=params['frozen']
    epochs=params['epochs']
    patience=params['patience']
    lr=params['lr']
    dropout_rate=params['dropout_rate']
    
    #
    cnn_by_own(num_class=5, frozen=frozen,epochs=epochs,patience=patience, lr=lr, dropout_rate=dropout_rate)
    #
    acc=prediction(cnn_model=load_model('./model_own.h5'))

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### 超参数优化BO-TPE

In [24]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()



  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.888030888030888                            
 10%|█         | 1/10 [02:06<19:02, 126.91s/trial, best loss: -0.888030888030888]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.888030888030888                                                       
 20%|██        | 2/10 [04:12<16:51, 126.39s/trial, best loss: -0.888030888030888]

...

accuracy:0.9787644787644788                                                       
 80%|████████  | 8/10 [25:54<07:27, 223.99s/trial, best loss: -0.9922779922779923]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.944015444015444                                                        
 90%|█████████ | 9/10 [29:29<03:41, 221.05s/trial, best loss: -0.9922779922779923]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9343629343629344                                                       
100%|██████████| 10/10 [32:45<00:00, 196.57s/trial, best loss: -0.9922779922779923]
Hyperopt estimated optimum {'dropout_rate': 0.5, 'epochs': 15.0, 'frozen': 17.0, 'lr': 0.005, 'patience': 3.0}


### 用优化后的超参数

In [29]:

params=get_params(format(best))

index=clear_output_df(index)
# Retrain the model by using the best hyperparameter values to obtain the best model
temptime=t2-t1

time_opt.append(temptime)
opt_type.append("BO-TPE")

t1=time.time()
cnn_by_own(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
17/17 [==============================] - 19s 1s/step - loss: 1.1987 - accuracy: 0.6224 - val_loss: 1.0022 - val_accuracy: 0.6834

Epoch 00001: val_accuracy improved from -inf to 0.68340, saving model to ./model_own.h5
Epoch 2/15
17/17 [==============================] - 19s 1s/step - loss: 0.9966 - accuracy: 0.6599 - val_loss: 0.8110 - val_accuracy: 0.6834

Epoch 00002: val_accuracy did not improve from 0.68340
Epoch 3/15
17/17 [==============================] - 17s 1s/step - loss: 0.8386 - accuracy: 0.7168 - val_loss: 0.5036 - val_accuracy: 0.7876

Epoch 00003: val_accuracy improved from 0.68340 to 0.78764, saving model to ./model_own.h5
Epoch 4/15
17/17 [==============================] - 17s 1s/step - loss: 0.4990 - accuracy: 0.8377 - val_loss: 0.2813 - val_accuracy: 0.8880

Epoch 00004: val_accuracy improved from 0.78764 to 0.88803, saving model to ./model_own.h5
Epoch 5/15
17/17 [==============================] - 18s 1s/step - loss: 0.3244 - accuracy: 0.8738 - val_loss: 0

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [30]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "CNN BO-TPE",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


### Random Search

In [31]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best)) 
t2=time.time()


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9401544401544402                           
 10%|█         | 1/10 [03:54<35:06, 234.06s/trial, best loss: -0.9401544401544402]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9401544401544402                                                       
 20%|██        | 2/10 [07:30<29:49, 223.71s/trial, best loss: -0.9401544401544402]

...

accuracy:0.8976833976833977                                                       
 80%|████████  | 8/10 [27:02<06:21, 190.61s/trial, best loss: -0.9942084942084942]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.888030888030888                                                        
 90%|█████████ | 9/10 [29:08<02:50, 170.50s/trial, best loss: -0.9942084942084942]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9131274131274131                                                       
100%|██████████| 10/10 [32:09<00:00, 192.97s/trial, best loss: -0.9942084942084942]
Hyperopt estimated optimum {'dropout_rate': 0.4, 'epochs': 20.0, 'frozen': 17.0, 'lr': 0.005, 'patience': 4.0}


### 用优化后的超参数

In [32]:
params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1

time_opt.append(temptime)
opt_type.append("RS")

t1=time.time()
cnn_by_own(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 19s 1s/step - loss: 1.1787 - accuracy: 0.6532 - val_loss: 0.9894 - val_accuracy: 0.6834

Epoch 00001: val_accuracy improved from -inf to 0.68340, saving model to ./model_own.h5
Epoch 2/20
17/17 [==============================] - 17s 970ms/step - loss: 0.9963 - accuracy: 0.6599 - val_loss: 0.9454 - val_accuracy: 0.6834

Epoch 00002: val_accuracy did not improve from 0.68340
Epoch 3/20
17/17 [==============================] - 19s 1s/step - loss: 0.7835 - accuracy: 0.7182 - val_loss: 0.5034 - val_accuracy: 0.8784

Epoch 00003: val_accuracy improved from 0.68340 to 0.87838, saving model to ./model_own.h5
Epoch 4/20
17/17 [==============================] - 18s 1s/step - loss: 0.5243 - accuracy: 0.8276 - val_loss: 0.4254 - val_accuracy: 0.8205

Epoch 00004: val_accuracy did not improve from 0.87838
Epoch 5/20
17/17 [==============================] - 18s 1s/step - loss: 0.4345 - accuracy: 0.8589 - val_loss: 0.2661 - val_accuracy: 0.8880

Epo

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [33]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "CNN RS",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


In [40]:
output_index

['CNN',
 'Xception',
 'VGG16',
 'VGG19',
 'Resnet',
 'Inception',
 'InceptionResnet',
 'CNN',
 'CNN',
 'VGG16']

# VGG16


In [38]:
def vgg16(num_class,epochs=20,frozen=15,lr=0.001,patience=2, dropout_rate=0.5,verbose=0, savepath='./VGG16.h5',history=history_VGG16,input_shape=INPUT_SIZE):
    model_fine_tune = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[earlyStopping, saveBestModel, history],
        verbose = verbose
    )
    return hist

In [41]:
def prediction(vgg_model):
#read images from validation folder
    rootdir = './test_224/'
    test_laels = []
    test_images=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
                continue
            test_laels.append(subdir.split('/')[-1])
            test_images.append(os.path.join(subdir, file))

    predict=[]
    length=len(test_images)
    label=validation_generator.class_indices
    label={v: k for k, v in label.items()}
    for i in range(length):
        inputimg=test_images[i]
        test_batch=[]
        thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
        #print(thisimg)
        test_shape=(1,)+thisimg.shape
        thisimg=thisimg.reshape(test_shape)
        avgg_model_batch=vgg_model.predict(thisimg) #use master model to process the input image
        #generate result by model 1
        prob=vgg_model_batch[0,np.argmax(vgg_model_batch,axis=1)[0]]
        res=label[np.argmax(vgg_model_batch,axis=1)[0]]
        predict.append(res)
    acc=accuracy_score(test_laels,predict)
    return acc

In [42]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'lr': abs(float(params['lr'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    frozen=params['frozen']
    epochs=params['epochs']
    patience=params['patience']
    lr=params['lr']
    dropout_rate=params['dropout_rate']
    
    #
    vgg16(num_class=5, frozen=frozen,epochs=epochs,patience=patience, lr=lr, dropout_rate=dropout_rate)
    #
    acc=prediction(vgg_model=load_model('./VGG16.h5'))

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### 超参数优化BO-TPE

In [43]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()



  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                          
 10%|█         | 1/10 [02:06<18:54, 126.03s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
 20%|██        | 2/10 [04:29<18:09, 136.24s/trial, best loss: -1.0]

...

accuracy:1.0                                                       
 80%|████████  | 8/10 [22:11<06:13, 186.96s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
 90%|█████████ | 9/10 [25:29<03:10, 190.35s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
100%|██████████| 10/10 [28:31<00:00, 171.15s/trial, best loss: -1.0]
Hyperopt estimated optimum {'dropout_rate': 0.4, 'epochs': 5.0, 'frozen': 15.0, 'lr': 0.001, 'patience': 4.0}


### 用优化后的超参数

In [44]:

params=get_params(format(best))

index=clear_output_df(index)
# Retrain the model by using the best hyperparameter values to obtain the best model
temptime=t2-t1

time_opt.append(temptime)
opt_type.append("BO-TPE")

t1=time.time()
vgg16(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/5
17/17 [==============================] - 20s 1s/step - loss: 1.0258 - accuracy: 0.6156 - val_loss: 0.3240 - val_accuracy: 0.8861

Epoch 00001: val_accuracy improved from -inf to 0.88610, saving model to ./VGG16.h5
Epoch 2/5
17/17 [==============================] - 17s 1s/step - loss: 0.3253 - accuracy: 0.8637 - val_loss: 0.2225 - val_accuracy: 0.8880

Epoch 00002: val_accuracy improved from 0.88610 to 0.88803, saving model to ./VGG16.h5
Epoch 3/5
17/17 [==============================] - 18s 1s/step - loss: 0.1674 - accuracy: 0.9273 - val_loss: 0.0105 - val_accuracy: 1.0000

Epoch 00003: val_accuracy improved from 0.88803 to 1.00000, saving model to ./VGG16.h5
Epoch 4/5
17/17 [==============================] - 18s 1s/step - loss: 0.0777 - accuracy: 0.9740 - val_loss: 0.0502 - val_accuracy: 0.9846

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/5
17/17 [==============================] - 18s 1s/step - loss: 0.0221 - accuracy: 0.9908 - val_loss: 0.0012 - val_accur

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [45]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "VGG16 BO-TPE",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


### 超参数优化Random Search

In [46]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                          
 10%|█         | 1/10 [03:36<32:28, 216.55s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
 20%|██        | 2/10 [05:43<21:48, 163.61s/trial, best loss: -1.0]

...

accuracy:1.0                                                       
 80%|████████  | 8/10 [18:47<04:34, 137.17s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
 90%|█████████ | 9/10 [20:53<02:13, 133.50s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9980694980694981                                        
100%|██████████| 10/10 [25:25<00:00, 152.51s/trial, best loss: -1.0]
Hyperopt estimated optimum {'dropout_rate': 0.6000000000000001, 'epochs': 20.0, 'frozen': 16.0, 'lr': 0.005, 'patience': 4.0}


### 用优化后的超参数


In [47]:
params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1

time_opt.append(temptime)
opt_type.append("RS")

t1=time.time()
vgg16(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
17/17 [==============================] - 20s 1s/step - loss: 9.5467 - accuracy: 0.5241 - val_loss: 0.9564 - val_accuracy: 0.6834

Epoch 00001: val_accuracy improved from -inf to 0.68340, saving model to ./VGG16.h5
Epoch 2/20
17/17 [==============================] - 17s 1s/step - loss: 0.6857 - accuracy: 0.6816 - val_loss: 0.4309 - val_accuracy: 0.7876

Epoch 00002: val_accuracy improved from 0.68340 to 0.78764, saving model to ./VGG16.h5
Epoch 3/20
17/17 [==============================] - 17s 1s/step - loss: 0.3765 - accuracy: 0.8406 - val_loss: 0.1992 - val_accuracy: 0.9208

Epoch 00003: val_accuracy improved from 0.78764 to 0.92085, saving model to ./VGG16.h5
Epoch 4/20
17/17 [==============================] - 18s 1s/step - loss: 0.3269 - accuracy: 0.8772 - val_loss: 0.1698 - val_accuracy: 0.9402

Epoch 00004: val_accuracy improved from 0.92085 to 0.94015, saving model to ./VGG16.h5
Epoch 5/20
17/17 [==============================] - 19s 1s/step - loss: 0.1950 - accuracy: 

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [48]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "VGG16 RS",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


# VGG19

In [49]:
def vgg19(num_class,epochs=20,frozen=15,lr=0.001,patience=2, dropout_rate=0.5,verbose=0, savepath='./VGG19.h5',history=history_VGG19,input_shape=INPUT_SIZE):
    model_fine_tune = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[earlyStopping, saveBestModel, history],
        verbose = verbose
    )
    return hist


In [50]:
def prediction(vgg_model):
#read images from validation folder
    rootdir = './test_224/'
    test_laels = []
    test_images=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
                continue
            test_laels.append(subdir.split('/')[-1])
            test_images.append(os.path.join(subdir, file))

    predict=[]
    length=len(test_images)
    label=validation_generator.class_indices
    label={v: k for k, v in label.items()}
    for i in range(length):
        inputimg=test_images[i]
        test_batch=[]
        thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
        #print(thisimg)
        test_shape=(1,)+thisimg.shape
        thisimg=thisimg.reshape(test_shape)
        vgg_model_batch=vgg_model.predict(thisimg) #use master model to process the input image
        #generate result by model 1
        prob=vgg_model_batch[0,np.argmax(vgg_model_batch,axis=1)[0]]
        res=label[np.argmax(vgg_model_batch,axis=1)[0]]
        predict.append(res)
    acc=accuracy_score(test_laels,predict)
    return acc

In [51]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'lr': abs(float(params['lr'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    frozen=params['frozen']
    epochs=params['epochs']
    patience=params['patience']
    lr=params['lr']
    dropout_rate=params['dropout_rate']

    vgg19(num_class=5, frozen=frozen,epochs=epochs,patience=patience, lr=lr, dropout_rate=dropout_rate)

    acc=prediction(vgg_model=load_model('./VGG19.h5'))

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### 超参数优化 BO-TPE

In [52]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9324324324324325                           
 10%|█         | 1/10 [03:16<29:32, 196.98s/trial, best loss: -0.9324324324324325]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.7876447876447876                                                       
 20%|██        | 2/10 [05:58<23:29, 176.17s/trial, best loss: -0.9324324324324325]

...

accuracy:0.6853281853281853                                        
 80%|████████  | 8/10 [21:18<05:25, 162.96s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.8861003861003861                                        
 90%|█████████ | 9/10 [23:25<02:31, 151.99s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.888030888030888                                         
100%|██████████| 10/10 [25:49<00:00, 154.92s/trial, best loss: -1.0]
Hyperopt estimated optimum {'dropout_rate': 0.30000000000000004, 'epochs': 15.0, 'frozen': 18.0, 'lr': 0.001, 'patience': 3.0}


### 用优化后的超参数

In [53]:

params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("BO-TPE")

t1=time.time()
vgg19(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/15
17/17 [==============================] - 20s 1s/step - loss: 1.1516 - accuracy: 0.6869 - val_loss: 0.7172 - val_accuracy: 0.7876

Epoch 00001: val_accuracy improved from -inf to 0.78764, saving model to ./VGG19.h5
Epoch 2/15
17/17 [==============================] - 18s 1s/step - loss: 0.5162 - accuracy: 0.8179 - val_loss: 0.3487 - val_accuracy: 0.7915

Epoch 00002: val_accuracy improved from 0.78764 to 0.79151, saving model to ./VGG19.h5
Epoch 3/15
17/17 [==============================] - 17s 1s/step - loss: 0.3032 - accuracy: 0.8579 - val_loss: 0.1832 - val_accuracy: 0.9093

Epoch 00003: val_accuracy improved from 0.79151 to 0.90927, saving model to ./VGG19.h5
Epoch 4/15
17/17 [==============================] - 18s 1s/step - loss: 0.2090 - accuracy: 0.9215 - val_loss: 0.0770 - val_accuracy: 0.9556

Epoch 00004: val_accuracy improved from 0.90927 to 0.95560, saving model to ./VGG19.h5
Epoch 5/15
17/17 [==============================] - 18s 1s/step - loss: 0.1518 - accuracy: 

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [54]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "VGG19 BO-TPE",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


### 超参数优化Random Search

In [55]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                          
 10%|█         | 1/10 [03:37<32:40, 217.80s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9942084942084942                                        
 20%|██        | 2/10 [07:14<28:56, 217.04s/trial, best loss: -1.0]

...

accuracy:1.0                                                       
 80%|████████  | 8/10 [25:37<06:57, 208.83s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9459459459459459                                        
 90%|█████████ | 9/10 [28:20<03:14, 194.33s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9980694980694981                                        
100%|██████████| 10/10 [33:26<00:00, 200.68s/trial, best loss: -1.0]
Hyperopt estimated optimum {'dropout_rate': 0.4, 'epochs': 10.0, 'frozen': 17.0, 'lr': 0.002, 'patience': 3.0}


### 用优化后的超参数

In [56]:

params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("RS")

t1=time.time()
vgg19(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
17/17 [==============================] - 19s 1s/step - loss: 3.0056 - accuracy: 0.4624 - val_loss: 1.1078 - val_accuracy: 0.6834

Epoch 00001: val_accuracy improved from -inf to 0.68340, saving model to ./VGG19.h5
Epoch 2/10
17/17 [==============================] - 18s 1s/step - loss: 1.2232 - accuracy: 0.6541 - val_loss: 1.0767 - val_accuracy: 0.6834

Epoch 00002: val_accuracy did not improve from 0.68340
Epoch 3/10
17/17 [==============================] - 18s 1s/step - loss: 1.0387 - accuracy: 0.6599 - val_loss: 0.6470 - val_accuracy: 0.6834

Epoch 00003: val_accuracy did not improve from 0.68340
Epoch 4/10
17/17 [==============================] - 18s 1s/step - loss: 0.6024 - accuracy: 0.6753 - val_loss: 0.5004 - val_accuracy: 0.7876

Epoch 00004: val_accuracy improved from 0.68340 to 0.78764, saving model to ./VGG19.h5
Epoch 5/10
17/17 [==============================] - 19s 1s/step - loss: 0.5702 - accuracy: 0.7139 - val_loss: 0.5118 - val_accuracy: 0.7838

Epoch 00005: v

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [57]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "VGG19 RS",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


# Xception

In [58]:
def xception( num_class,epochs=20,frozen=15,lr=0.001,patience=2, dropout_rate=0.5,verbose=0,savepath='./xception.h5',history=history_this,input_shape=INPUT_SIZE):
    model_fine_tune = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:		#could be tuned to be 50, 100, or 131
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='xception')
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto')	#patience could be tuned by 2 and 3
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        verbose=verbose,
        callbacks=[earlyStopping, saveBestModel, history],
    )



In [64]:
def prediction(xception_model):
#read images from validation folder
    rootdir = './test_224/'
    test_laels = []
    test_images=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
                continue
            test_laels.append(subdir.split('/')[-1])
            test_images.append(os.path.join(subdir, file))

    predict=[]
    length=len(test_images)
    label=validation_generator.class_indices
    label={v: k for k, v in label.items()}
    for i in range(length):
        inputimg=test_images[i]
        test_batch=[]
        thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
        #print(thisimg)
        test_shape=(1,)+thisimg.shape
        thisimg=thisimg.reshape(test_shape)
        xception_model_batch=xception_model.predict(thisimg) #use master model to process the input image
        #generate result by model 1
        #
        prob=xception_model_batch[0,np.argmax(xception_model_batch,axis=1)[0]]
        res=label[np.argmax(xception_model_batch,axis=1)[0]]
        predict.append(res)
    acc=accuracy_score(test_laels,predict)
    return acc

In [65]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'lr': abs(float(params['lr'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    frozen=params['frozen']
    epochs=params['epochs']
    patience=params['patience']
    lr=params['lr']
    dropout_rate=params['dropout_rate']
    
    #
    xception(num_class=5, frozen=frozen,epochs=epochs,patience=patience, lr=lr, dropout_rate=dropout_rate)
    #
    acc=prediction(xception_model=load_model('./xception.h5'))

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### 超参数优化 BO-TPE

In [66]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                          
 10%|█         | 1/10 [03:10<28:38, 190.94s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
 20%|██        | 2/10 [05:25<21:04, 158.05s/trial, best loss: -1.0]

...

accuracy:1.0                                                       
 80%|████████  | 8/10 [21:12<05:10, 155.19s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
 90%|█████████ | 9/10 [24:17<02:44, 164.61s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
100%|██████████| 10/10 [26:44<00:00, 160.50s/trial, best loss: -1.0]
Hyperopt estimated optimum {'dropout_rate': 0.4, 'epochs': 10.0, 'frozen': 16.0, 'lr': 0.004, 'patience': 3.0}


### 用优化后的超参数

In [67]:

params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("BO-TPE")

t1=time.time()
xception(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
17/17 [==============================] - 21s 1s/step - loss: 0.5744 - accuracy: 0.7948 - val_loss: 0.1081 - val_accuracy: 0.9768

Epoch 00001: val_accuracy improved from -inf to 0.97683, saving model to ./xception.h5


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
17/17 [==============================] - 17s 1s/step - loss: 0.1010 - accuracy: 0.9754 - val_loss: 0.0386 - val_accuracy: 0.9903

Epoch 00002: val_accuracy improved from 0.97683 to 0.99035, saving model to ./xception.h5
Epoch 3/10
17/17 [==============================] - 18s 1s/step - loss: 0.0490 - accuracy: 0.9908 - val_loss: 0.0216 - val_accuracy: 1.0000

Epoch 00003: val_accuracy improved from 0.99035 to 1.00000, saving model to ./xception.h5
Epoch 4/10
17/17 [==============================] - 15s 894ms/step - loss: 0.0308 - accuracy: 0.9966 - val_loss: 0.0154 - val_accuracy: 0.9981

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/10
17/17 [==============================] - 18s 1s/step - loss: 0.0262 - accuracy: 0.9942 - val_loss: 0.0118 - val_accuracy: 1.0000

Epoch 00005: val_accuracy did not improve from 1.00000
Epoch 6/10
17/17 [==============================] - 18s 1s/step - loss: 0.0183 - accuracy: 0.9957 - val_loss: 0.0094 - val_accuracy: 1.0000

Ep

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [68]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "Xception BO-TPE",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


### 超参数优化Random Search

In [69]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9942084942084942                           
 10%|█         | 1/10 [02:11<19:43, 131.51s/trial, best loss: -0.9942084942084942]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9903474903474904                                                       
 20%|██        | 2/10 [04:06<16:15, 121.88s/trial, best loss: -0.9942084942084942]

...

accuracy:1.0                                                       
 80%|████████  | 8/10 [19:33<04:53, 146.80s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
 90%|█████████ | 9/10 [22:56<02:44, 164.24s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                       
100%|██████████| 10/10 [25:43<00:00, 154.30s/trial, best loss: -1.0]
Hyperopt estimated optimum {'dropout_rate': 0.6000000000000001, 'epochs': 10.0, 'frozen': 15.0, 'lr': 0.006, 'patience': 3.0}


### 用优化后的超参数

In [70]:

params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("RS")

t1=time.time()
xception(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
17/17 [==============================] - 20s 1s/step - loss: 0.5457 - accuracy: 0.8280 - val_loss: 0.1022 - val_accuracy: 0.9846

Epoch 00001: val_accuracy improved from -inf to 0.98456, saving model to ./xception.h5


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
17/17 [==============================] - 18s 1s/step - loss: 0.1066 - accuracy: 0.9692 - val_loss: 0.0368 - val_accuracy: 0.9942

Epoch 00002: val_accuracy improved from 0.98456 to 0.99421, saving model to ./xception.h5
Epoch 3/10
17/17 [==============================] - 18s 1s/step - loss: 0.0502 - accuracy: 0.9928 - val_loss: 0.0222 - val_accuracy: 0.9961

Epoch 00003: val_accuracy improved from 0.99421 to 0.99614, saving model to ./xception.h5
Epoch 4/10
17/17 [==============================] - 18s 1s/step - loss: 0.0304 - accuracy: 0.9947 - val_loss: 0.0139 - val_accuracy: 1.0000

Epoch 00004: val_accuracy improved from 0.99614 to 1.00000, saving model to ./xception.h5
Epoch 5/10
17/17 [==============================] - 18s 1s/step - loss: 0.0202 - accuracy: 0.9971 - val_loss: 0.0107 - val_accuracy: 1.0000

Epoch 00005: val_accuracy did not improve from 1.00000
Epoch 6/10
17/17 [==============================] - 18s 1s/step - loss: 0.0160 - accuracy: 0.9990 - val_loss: 0

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [71]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "Xception RS",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


# Inception

In [72]:
def inception(num_class,epochs=20,frozen=15,lr=0.001,patience=2, dropout_rate=0.5,verbose=0,  savepath='./inception.h5',history=history_Inception,input_shape=INPUT_SIZE):
    model_fine_tune = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
        verbose=verbose
    )

In [73]:
def prediction(inception_model):
#read images from validation folder
    rootdir = './test_224/'
    test_laels = []
    test_images=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
                continue
            test_laels.append(subdir.split('/')[-1])
            test_images.append(os.path.join(subdir, file))

    predict=[]
    length=len(test_images)
    label=validation_generator.class_indices
    label={v: k for k, v in label.items()}
    for i in range(length):
        inputimg=test_images[i]
        test_batch=[]
        thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
        #print(thisimg)
        test_shape=(1,)+thisimg.shape
        thisimg=thisimg.reshape(test_shape)
        inception_model_batch=inception_model.predict(thisimg) #use master model to process the input image
        #generate result by model 1
        #
        prob=inception_model_batch[0,np.argmax(inception_model_batch,axis=1)[0]]
        res=label[np.argmax(inception_model_batch,axis=1)[0]]
        predict.append(res)
    acc=accuracy_score(test_laels,predict)
    return acc






In [74]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'lr': abs(float(params['lr'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    frozen=params['frozen']
    epochs=params['epochs']
    patience=params['patience']
    lr=params['lr']
    dropout_rate=params['dropout_rate']
    
    #
    inception(num_class=5, frozen=frozen,epochs=epochs,patience=patience, lr=lr, dropout_rate=dropout_rate)
    #
    acc=prediction(inception_model=load_model('./inception.h5'))

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }

In [75]:
    #Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                          
 10%|█         | 1/10 [04:00<36:03, 240.42s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.6833976833976834                                        
 20%|██        | 2/10 [06:09<23:18, 174.78s/trial, best loss: -1.0]

...

accuracy:0.6486486486486487                                        
 80%|████████  | 8/10 [25:41<07:10, 215.50s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9961389961389961                                        
 90%|█████████ | 9/10 [29:02<03:31, 211.12s/trial, best loss: -1.0]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.10424710424710425                                       
100%|██████████| 10/10 [31:07<00:00, 186.78s/trial, best loss: -1.0]
Hyperopt estimated optimum {'dropout_rate': 0.30000000000000004, 'epochs': 10.0, 'frozen': 15.0, 'lr': 0.004, 'patience': 3.0}


In [76]:

params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("BO-TPE")

t1=time.time()
inception(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
17/17 [==============================] - 24s 1s/step - loss: 0.2464 - accuracy: 0.9162 - val_loss: 3.0204 - val_accuracy: 0.7876

Epoch 00001: val_accuracy improved from -inf to 0.78764, saving model to ./inception.h5
Epoch 2/10
17/17 [==============================] - 18s 1s/step - loss: 0.0974 - accuracy: 0.9851 - val_loss: 171.0232 - val_accuracy: 0.6834

Epoch 00002: val_accuracy did not improve from 0.78764
Epoch 3/10
17/17 [==============================] - 18s 1s/step - loss: 0.0076 - accuracy: 0.9981 - val_loss: 30.6622 - val_accuracy: 0.7104

Epoch 00003: val_accuracy did not improve from 0.78764
Epoch 4/10
17/17 [==============================] - 18s 1s/step - loss: 0.0173 - accuracy: 0.9957 - val_loss: 48.5291 - val_accuracy: 0.7857

Epoch 00004: val_accuracy did not improve from 0.78764
Epoch 00004: early stopping


/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [77]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "inception BO-TPE",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


In [78]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.694980694980695                            
 10%|█         | 1/10 [02:08<19:12, 128.10s/trial, best loss: -0.694980694980695]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.915057915057915                                                       
 20%|██        | 2/10 [04:48<19:38, 147.31s/trial, best loss: -0.915057915057915]

...

accuracy:0.38223938223938225                                                     
 80%|████████  | 8/10 [21:09<05:23, 161.93s/trial, best loss: -0.915057915057915]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9111969111969112                                                      
 90%|█████████ | 9/10 [25:15<03:07, 187.97s/trial, best loss: -0.915057915057915]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                                     
100%|██████████| 10/10 [30:35<00:00, 183.57s/trial, best loss: -1.0]             
Hyperopt estimated optimum {'dropout_rate': 0.6000000000000001, 'epochs': 15.0, 'frozen': 16.0, 'lr': 0.004, 'patience': 3.0}


In [79]:
params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("RS")

t1=time.time()
inception(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/15
17/17 [==============================] - 26s 1s/step - loss: 0.1617 - accuracy: 0.9316 - val_loss: 6.7961 - val_accuracy: 0.7857

Epoch 00001: val_accuracy improved from -inf to 0.78571, saving model to ./inception.h5
Epoch 2/15
17/17 [==============================] - 19s 1s/step - loss: 0.0028 - accuracy: 0.9990 - val_loss: 0.6612 - val_accuracy: 0.8842

Epoch 00002: val_accuracy improved from 0.78571 to 0.88417, saving model to ./inception.h5
Epoch 3/15
17/17 [==============================] - 20s 1s/step - loss: 0.0036 - accuracy: 0.9986 - val_loss: 24.5681 - val_accuracy: 0.1564

Epoch 00003: val_accuracy did not improve from 0.88417
Epoch 4/15
17/17 [==============================] - 19s 1s/step - loss: 0.0021 - accuracy: 0.9990 - val_loss: 24.0498 - val_accuracy: 0.2124

Epoch 00004: val_accuracy did not improve from 0.88417
Epoch 5/15
17/17 [==============================] - 18s 1s/step - loss: 2.3830e-04 - accuracy: 1.0000 - val_loss: 14.6312 - val_accuracy: 0.3707


/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [80]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "inception RS",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


In [81]:
output_df

,Loss,Accuracy,Val-Loss,Val-Accuracy,Time
0,1.381341,0.634875,1.066410,0.683398,NaN
1,0.025501,0.999037,0.013141,1.000000,NaN
2,0.021232,0.995183,0.000791,1.000000,NaN
3,0.139370,0.942197,0.011500,1.000000,NaN
4,0.594463,0.800096,87.009918,0.104247,NaN
5,0.183305,0.932081,2.650600,0.683398,NaN
6,0.020698,0.995183,3.605927,0.789575,NaN
7,0.102476,0.956166,0.039737,0.986486,NaN
8,0.111195,0.958093,0.039620,1.000000,NaN
9,0.167389,0.927264,0.010488,1.000000,NaN


In [82]:
output_index

['CNN',
 'Xception',
 'VGG16',
 'VGG19',
 'Resnet',
 'Inception',
 'InceptionResnet',
 'CNN',
 'CNN',
 'VGG16',
 'VGG16',
 'VGG19',
 'VGG19',
 'Xception',
 'Xception',
 'Inception',
 'Inception']

# Resnet

In [83]:
def resnet( num_class,epochs=20,frozen=15,lr=0.001,patience=2, dropout_rate=0.5,verbose=0,  savepath='./resnet.h5',history=history_Resnet,input_shape=INPUT_SIZE):
    model_fine_tune = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
        verbose=verbose
    )

In [84]:
def prediction(resnet_model):
#read images from validation folder
    rootdir = './test_224/'
    test_laels = []
    test_images=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
                continue
            test_laels.append(subdir.split('/')[-1])
            test_images.append(os.path.join(subdir, file))

    predict=[]
    length=len(test_images)
    label=validation_generator.class_indices
    label={v: k for k, v in label.items()}
    for i in range(length):
        inputimg=test_images[i]
        test_batch=[]
        thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
        #print(thisimg)
        test_shape=(1,)+thisimg.shape
        thisimg=thisimg.reshape(test_shape)
        resnet_model_batch=resnet_model.predict(thisimg) #use master model to process the input image
        #generate result by model 1
        #
        prob=resnet_model_batch[0,np.argmax(resnet_model_batch,axis=1)[0]]
        res=label[np.argmax(resnet_model_batch,axis=1)[0]]
        predict.append(res)
    acc=accuracy_score(test_laels,predict)
    return acc





In [85]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'lr': abs(float(params['lr'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    frozen=params['frozen']
    epochs=params['epochs']
    patience=params['patience']
    lr=params['lr']
    dropout_rate=params['dropout_rate']
    
    #
    resnet(num_class=5, frozen=frozen,epochs=epochs,patience=patience, lr=lr, dropout_rate=dropout_rate)
    #
    acc=prediction(resnet_model=load_model('./resnet.h5'))

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }

In [86]:
    #Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.7876447876447876                           
 10%|█         | 1/10 [02:00<18:06, 120.71s/trial, best loss: -0.7876447876447876]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9478764478764479                                                       
 20%|██        | 2/10 [05:42<24:00, 180.02s/trial, best loss: -0.9478764478764479]

...

accuracy:0.8455598455598455                                                       
 80%|████████  | 8/10 [19:49<05:05, 152.64s/trial, best loss: -0.9478764478764479]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.7818532818532818                                                       
 90%|█████████ | 9/10 [22:51<02:41, 161.83s/trial, best loss: -0.9478764478764479]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.10424710424710425                                                      
100%|██████████| 10/10 [25:11<00:00, 151.19s/trial, best loss: -0.9478764478764479]
Hyperopt estimated optimum {'dropout_rate': 0.6000000000000001, 'epochs': 10.0, 'frozen': 15.0, 'lr': 0.005, 'patience': 3.0}


In [87]:
params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("BO-TPE")

t1=time.time()
resnet(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
17/17 [==============================] - 24s 1s/step - loss: 0.5453 - accuracy: 0.8276 - val_loss: 6.8200 - val_accuracy: 0.6834

Epoch 00001: val_accuracy improved from -inf to 0.68340, saving model to ./resnet.h5


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
17/17 [==============================] - 19s 1s/step - loss: 0.1328 - accuracy: 0.9422 - val_loss: 12.7873 - val_accuracy: 0.7162

Epoch 00002: val_accuracy improved from 0.68340 to 0.71622, saving model to ./resnet.h5
Epoch 3/10
17/17 [==============================] - 19s 1s/step - loss: 0.0661 - accuracy: 0.9807 - val_loss: 276.2966 - val_accuracy: 0.2046

Epoch 00003: val_accuracy did not improve from 0.71622
Epoch 4/10
17/17 [==============================] - 16s 940ms/step - loss: 0.0313 - accuracy: 0.9913 - val_loss: 1089.2363 - val_accuracy: 0.1004

Epoch 00004: val_accuracy did not improve from 0.71622
Epoch 5/10
17/17 [==============================] - 18s 1s/step - loss: 0.0136 - accuracy: 0.9971 - val_loss: 537.2593 - val_accuracy: 0.1004

Epoch 00005: val_accuracy did not improve from 0.71622
Epoch 00005: early stopping


/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [88]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "resnet BO-TPE",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


In [89]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
###

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.10424710424710425                          
 10%|█         | 1/10 [02:22<21:24, 142.75s/trial, best loss: -0.10424710424710425]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.20463320463320464                                                       
 20%|██        | 2/10 [05:01<20:16, 152.08s/trial, best loss: -0.20463320463320464]

...

accuracy:0.10424710424710425                                                      
 80%|████████  | 8/10 [19:03<04:51, 145.78s/trial, best loss: -0.7857142857142857]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.8687258687258688                                                       
 90%|█████████ | 9/10 [21:28<02:25, 145.44s/trial, best loss: -0.8687258687258688]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.7876447876447876                                                       
100%|██████████| 10/10 [24:09<00:00, 144.97s/trial, best loss: -0.8687258687258688]
Hyperopt estimated optimum {'dropout_rate': 0.5, 'epochs': 20.0, 'frozen': 16.0, 'lr': 0.005, 'patience': 3.0}


In [90]:
params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("RS")

t1=time.time()
resnet(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
17/17 [==============================] - 23s 1s/step - loss: 0.6197 - accuracy: 0.7991 - val_loss: 36.2834 - val_accuracy: 0.1042

Epoch 00001: val_accuracy improved from -inf to 0.10425, saving model to ./resnet.h5


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
17/17 [==============================] - 19s 1s/step - loss: 0.2160 - accuracy: 0.9056 - val_loss: 220.5841 - val_accuracy: 0.1042

Epoch 00002: val_accuracy did not improve from 0.10425
Epoch 3/20
17/17 [==============================] - 19s 1s/step - loss: 0.1179 - accuracy: 0.9470 - val_loss: 113.0140 - val_accuracy: 0.2046

Epoch 00003: val_accuracy improved from 0.10425 to 0.20463, saving model to ./resnet.h5
Epoch 4/20
17/17 [==============================] - 19s 1s/step - loss: 0.0898 - accuracy: 0.9658 - val_loss: 236.5030 - val_accuracy: 0.2046

Epoch 00004: val_accuracy did not improve from 0.20463
Epoch 5/20
17/17 [==============================] - 18s 1s/step - loss: 0.0319 - accuracy: 0.9928 - val_loss: 33.3170 - val_accuracy: 0.1274

Epoch 00005: val_accuracy did not improve from 0.20463
Epoch 6/20
17/17 [==============================] - 17s 996ms/step - loss: 0.0138 - accuracy: 0.9957 - val_loss: 165.7761 - val_accuracy: 0.1042

Epoch 00006: val_accuracy did 

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [91]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "resnet RS",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


# InceptionResnet

In [96]:
def inceptionresnet( num_class,epochs=20,frozen=15,lr=0.001,patience=2, dropout_rate=0.5,verbose=0,  savepath='./inceptionresnet.h5',history=history_InceptionResnet,input_shape=INPUT_SIZE):
    model_fine_tune = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 400-550
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
        verbose=verbose
    )

In [97]:
def prediction(inceptionresnet_model):
#read images from validation folder
    rootdir = './test_224/'
    test_laels = []
    test_images=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
                continue
            test_laels.append(subdir.split('/')[-1])
            test_images.append(os.path.join(subdir, file))

    predict=[]
    length=len(test_images)
    label=validation_generator.class_indices
    label={v: k for k, v in label.items()}
    for i in range(length):
        inputimg=test_images[i]
        test_batch=[]
        thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
        #print(thisimg)
        test_shape=(1,)+thisimg.shape
        thisimg=thisimg.reshape(test_shape)
        inceptionresnet_model_batch=inceptionresnet_model.predict(thisimg) #use master model to process the input image
        #generate result by model 1
        #
        prob=inceptionresnet_model_batch[0,np.argmax(inceptionresnet_model_batch,axis=1)[0]]
        res=label[np.argmax(inceptionresnet_model_batch,axis=1)[0]]
        predict.append(res)
    acc=accuracy_score(test_laels,predict)
    return acc




In [98]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'lr': abs(float(params['lr'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    frozen=params['frozen']
    epochs=params['epochs']
    patience=params['patience']
    lr=params['lr']
    dropout_rate=params['dropout_rate']
    
    #
    inceptionresnet(num_class=5, frozen=frozen,epochs=epochs,patience=patience, lr=lr, dropout_rate=dropout_rate)
    #
    acc=prediction(inceptionresnet_model=load_model('./inceptionresnet.h5'))

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }

In [99]:
    #Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.6833976833976834                           
 10%|█         | 1/10 [02:21<21:17, 141.98s/trial, best loss: -0.6833976833976834]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.8957528957528957                                                       
 20%|██        | 2/10 [05:46<23:49, 178.73s/trial, best loss: -0.8957528957528957]

...

accuracy:0.8474903474903475                                                       
 80%|████████  | 8/10 [24:33<05:40, 170.03s/trial, best loss: -0.9555984555984556]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.8397683397683398                                                       
 90%|█████████ | 9/10 [27:09<02:45, 165.75s/trial, best loss: -0.9555984555984556]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.9903474903474904                                                       
100%|██████████| 10/10 [31:54<00:00, 191.43s/trial, best loss: -0.9903474903474904]
Hyperopt estimated optimum {'dropout_rate': 0.5, 'epochs': 10.0, 'frozen': 16.0, 'lr': 0.005, 'patience': 3.0}


In [100]:

params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("BO-TPE")

t1=time.time()
inceptionresnet(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
17/17 [==============================] - 33s 1s/step - loss: 0.2660 - accuracy: 0.8969 - val_loss: 13.8054 - val_accuracy: 0.3842

Epoch 00001: val_accuracy improved from -inf to 0.38417, saving model to ./inceptionresnet.h5
Epoch 2/10
17/17 [==============================] - 19s 1s/step - loss: 0.0505 - accuracy: 0.9860 - val_loss: 444.5540 - val_accuracy: 0.1004

Epoch 00002: val_accuracy did not improve from 0.38417
Epoch 3/10
17/17 [==============================] - 18s 1s/step - loss: 0.0115 - accuracy: 0.9966 - val_loss: 52.6188 - val_accuracy: 0.4054

Epoch 00003: val_accuracy improved from 0.38417 to 0.40541, saving model to ./inceptionresnet.h5
Epoch 4/10
17/17 [==============================] - 19s 1s/step - loss: 0.0030 - accuracy: 0.9995 - val_loss: 25.7887 - val_accuracy: 0.2046

Epoch 00004: val_accuracy did not improve from 0.40541
Epoch 5/10
17/17 [==============================] - 17s 1s/step - loss: 2.1931e-04 - accuracy: 1.0000 - val_loss: 8.1681 - val_acc

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [101]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "inceptionresnet BO-TPE",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


In [102]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.8957528957528957                           
 10%|█         | 1/10 [03:22<30:19, 202.19s/trial, best loss: -0.8957528957528957]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.7451737451737451                                                       
 20%|██        | 2/10 [05:41<22:01, 165.20s/trial, best loss: -0.8957528957528957]

...

accuracy:0.6833976833976834                                                       
 80%|████████  | 8/10 [24:27<05:45, 172.90s/trial, best loss: -0.9845559845559846]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:0.803088803088803                                                        
 90%|█████████ | 9/10 [27:04<02:47, 167.97s/trial, best loss: -0.9845559845559846]

/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)



accuracy:1.0                                                                      
100%|██████████| 10/10 [32:20<00:00, 194.02s/trial, best loss: -1.0]              
Hyperopt estimated optimum {'dropout_rate': 0.6000000000000001, 'epochs': 20.0, 'frozen': 18.0, 'lr': 0.003, 'patience': 4.0}


In [103]:
params=get_params(format(best))
index=clear_output_df(index)

temptime=t2-t1
time_opt.append(temptime)
opt_type.append("RS")

t1=time.time()
inceptionresnet(**params)
t2=time.time()

temptime=t2-t1
timelist.append(temptime)


/root/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
17/17 [==============================] - 30s 1s/step - loss: 0.2576 - accuracy: 0.9041 - val_loss: 16.2143 - val_accuracy: 0.7085

Epoch 00001: val_accuracy improved from -inf to 0.70849, saving model to ./inceptionresnet.h5
Epoch 2/20
17/17 [==============================] - 18s 1s/step - loss: 0.0279 - accuracy: 0.9928 - val_loss: 276.3622 - val_accuracy: 0.7239

Epoch 00002: val_accuracy improved from 0.70849 to 0.72394, saving model to ./inceptionresnet.h5
Epoch 3/20
17/17 [==============================] - 18s 1s/step - loss: 0.0318 - accuracy: 0.9918 - val_loss: 32.9751 - val_accuracy: 0.6834

Epoch 00003: val_accuracy did not improve from 0.72394
Epoch 4/20
17/17 [==============================] - 18s 1s/step - loss: 9.8630e-04 - accuracy: 1.0000 - val_loss: 16.3054 - val_accuracy: 0.6873

Epoch 00004: val_accuracy did not improve from 0.72394
Epoch 5/20
17/17 [==============================] - 17s 1s/step - loss: 3.0373e-04 - accuracy: 1.0000 - val_loss: 6.4805 - val

/tmp/ipykernel_326/422981087.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [104]:
resp = requests.post("https://www.autodl.com/api/v1/wechat/message/push",
                     json={
                         "token": "444db7f1cca1",
                         "title": "inceptionresnet RS",
                         "name": "Omar Project"
                     })
print(resp.content.decode())

{"code":"Success","data":null,"msg":""}


In [105]:
output_df.index=output_index
output_df['Time']=timelist
output_df['Time_opt']=time_opt
output_df['Opt_type']=opt_type


In [106]:

# Save the result to file
import datetime
output_df.to_excel('result-{}.xlsx'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S')))